In [1]:
import torch

In [18]:
a = torch.rand(2,3,3,3,3)

In [24]:
mse = torch.sum(((a)**2), (2, 3, 4))

In [25]:
mse.shape

torch.Size([2, 3])

In [26]:
torch.mean(mse)

tensor(8.6029)

In [27]:
mse = torch.mean(((a))**2)

In [28]:
mse

tensor(0.3186)